<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/010_read.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Чтение данных в Pandas для ML


## Подготовка среды и тестов

In [ ]:
!pip install pandas numpy matplotlib seaborn pyarrow fastparquet scipy cudf

> ⚠️ How to install cudf for google colab:

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [3]:
import pandas as pd
import numpy as np

# Создание DataFrame с нуля с нужными столбцами
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3],
    'name': ['Иван', 'Петр', 'Анна', 'Мария', 'Сергей', 'Елена', 'Олег', 'Юлия', 'Дмитрий', 'Наталья', 'Петр', 'Анна'],
    'email': ['ivan@example.com', 'petr@example.com', np.nan, 'maria@example.com', 'sergey@example.com', 'elena@example.com', 'oleg@example.com', 'yulia@example.com', 'dmitry@example.com', 'natalya@example.com', 'petr@example.com', np.nan],
    'value': [100.5, 200.3, 150.0, np.nan, 300.1, 250.8, 180.0, 220.5, 280.9, 190.2, 200.3, 150.0],
    'category_col1': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A'],
    'date_col': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-02', '2023-01-03']),
    'city': ['Москва', 'Санкт-Петербург', 'Москва', 'Казань', 'Санкт-Петербург', 'Москва', 'Казань', 'Москва', 'Санкт-Петербург', 'Казань', 'Санкт-Петербург', 'Москва'],
    'target': [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0]
}

df_demo = pd.DataFrame(data)

# Сохранение DataFrame в CSV файл
df_demo.to_csv('data.csv', encoding='utf-8', index=False, na_rep='?')


## 🟢 Базовый уровень (Must Know для начинающих)

### 1.1 Чтение данных из CSV

In [4]:
import pandas as pd

# Простейший случай
df = pd.read_csv('data.csv')

# Сложнее: работа с "грязными" файлами
df = pd.read_csv(
    'data.csv',
    sep=',',                 # нестандартный разделитель
    encoding='utf-8',        # поддержка кириллицы или других кодировок
    na_values=['NA', '?'],   # обозначения пропусков
    parse_dates=['date_col'] # автоматический парсинг дат
)

> ⚠️ Если данные приходят без заголовков, используйте `header=None` и задайте названия колонок через `names`.

### 1.2 Просмотр данных

In [ ]:
print("Первые 5 строк:")
display(df.head())

print("\nОсновная статистика:")
display(df.describe(include='all'))  # include='all' покажет все типы

print("\nИнформация о типах:")
display(df.info())

> 💡 Для быстрого анализа можно использовать `pandas_profiling` или `sweetviz`.

### 1.3 Сохранение данных

In [6]:
df.to_csv('processed_data.csv', index=False)  # без индексов
df.to_excel('report.xlsx', sheet_name='Data')  # сохранение в Excel

> ✅ Всегда используйте `index=False`, если не нужно сохранять индекс.

### 1.4 Решение базовых проблем

In [7]:
# Удаление пропусков
df.dropna(subset=['email'], inplace=True)

# Заполнение пропусков медианой
df['value'] = pd.to_numeric(df['value'], errors='coerce') # некорректные значения превращаются в NaN
df.fillna({'value': df['value'].median()}, inplace=True)

# Удаление дубликатов
df = df.drop_duplicates()

> 💡 Можно заполнять пропуски и средним, и модой (`mode()`), в зависимости от типа данных.

## 🟡 Продвинутый уровень (Для опытных ML-инженеров)

### 2.1 Оптимизация памяти

In [ ]:
def optimize_dtypes(df):
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

df = optimize_dtypes(df)
print(f"Память: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

> 📈 Такая оптимизация особенно важна на этапе предобработки больших датасетов.

### 2.2 Работа с категориальными признаками

In [ ]:
# Преобразование в категориальный тип
df['category_col1'] = df['category_col1'].astype('category')

# One-Hot Encoding
df = pd.get_dummies(df, columns=['city'], prefix='cat')

df.info()
df.head()

> ⚠️ Для бустинговых моделей (CatBoost, LightGBM) лучше оставлять как `category`.

### 2.3 Эффективное чтение больших данных

In [10]:
# Чтение только нужных столбцов
cols = ['id', 'name', 'email']
df_limited = pd.read_csv('data.csv', usecols=cols)

# Сохранение в Parquet (быстрее CSV в 5–10 раз)
df_limited.to_parquet('data.parquet', engine='pyarrow')

> ✅ Используйте Parquet или Feather для хранения промежуточных результатов.

### 2.4 Интеграция с ML-библиотеками

In [11]:
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1).values # преобразуем в numpy array
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

> 💡 Для удобства работы можно оставить данные в виде `DataFrame` и использовать `.to_numpy()` при необходимости.

## 🔴 Экспертный уровень (Для хакатонов и Kaggle Grandmasters)

### 3.1 Работа с гигабайтными данными

In [12]:
chunk_size = 5  # 100k строк за раз
filtered_rows = []

for chunk in pd.read_csv('data.csv', chunksize=chunk_size):
    chunk = chunk[(chunk['category_col1']) == 'A']  # фильтрация на лету
    filtered_rows.append(chunk)

df = pd.concat(filtered_rows)

> 📦 Это позволяет работать с файлами, которые больше доступной оперативной памяти.

### 3.2 Разреженные матрицы для текстов/NLP

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz

vectorizer = TfidfVectorizer()
sparse_matrix = vectorizer.fit_transform(df['city'])

print(sparse_matrix)
# Сохранение в формате, совместимом с LibSVM
save_npz('sparse_data.npz', sparse_matrix)

> ✅ Разреженные матрицы экономят память и подходят для моделей вроде SVM, LogisticRegression.

### 3.3 GPU-ускорение с RAPIDS

In [14]:
import cudf

# Чтение данных на GPU
gdf = cudf.read_csv('data.csv', na_values='?')
gdf.dropna(inplace=True)
gdf = gdf.query("value > 200")  # фильтрация в 10–50x быстрее, чем pandas

# Конвертация в numpy для моделей
X = gdf.to_pandas().values


> 🚀 Требуется CUDA-совместимая видеокарта и установка библиотек RAPIDS.

### 3.4 Экстремальная оптимизация

In [15]:
# Уменьшение точности float64 -> float16
df = df.astype({col: 'float16' for col in df.select_dtypes(include=['float']).columns})

# Словарное сжатие для строк
df['city'] = df['city'].astype('category').cat.codes


> 🧪 Полезно для очень больших датасетов, где каждый байт важен.

### 3.5 Инкрементальное обучение (Online Learning)

In [16]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()
for chunk in pd.read_csv('data.csv', usecols=['id', 'target'], na_values="?", chunksize=5):
    X = chunk.drop('target', axis=1)
    y = chunk['target']
    model.partial_fit(X, y, classes=[0, 1])

> 🔄 Подходит для потоковой обработки данных, например, в реальном времени.

## 📊 Чеклист по уровням

| Уровень | Навыки |
|--------|--------|
| 🟢     | Чтение CSV/Excel, анализ данных, fillna/drop_duplicates |
| 🟡     | Оптимизация типов, категориальные признаки, parquet, train_test_split |
| 🔴     | Итеративная обработка, GPU-ускорение, разреженные матрицы, online learning |

## ⚠️ Антипаттерны

| Уровень | Что делать не стоит |
|---------|---------------------|
| 🟢      | Сохранять данные с индексом без необходимости |
| 🟢      | Читать весь большой файл целиком ради просмотра |
| 🟡      | Не использовать `usecols` при работе с большими файлами |
| 🔴      | Игнорировать `dtype` при чтении больших данных |
| 🔴      | Использовать медленные циклы вместо векторизованных операций |

## 🚀 Полезные советы

1. **Тестирование перед обработкой:**  
   ```python
   pd.read_csv('test_data.csv', nrows=10)  # проверка структуры
   ```

2. **Быстрая конвертация в NumPy:**  
   ```python
   X = df.values  # работает быстро
   ```

3. **Проверка памяти:**  
   ```python
   df.memory_usage(deep=True)
   ```

4. **Избежание копий:**  
   Используйте `inplace=True` или переприсваивайте переменные явно.

## 📌 Итог

Pandas — основной инструмент для подготовки данных в ML. Он прост в освоении, но мощный в использовании. От базового анализа до экспертных техник вроде GPU-ускорения и онлайн-обучения — он покрывает полный цикл работы с данными.

> 🎯 Главное правило: всегда помни, что ты готовишь данные **для модели**, поэтому делай это эффективно и аккуратно.